I will be using NLTK library and use VADER for the sentiment analysis!

In [1]:
!pip install nltk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [20]:
import nltk
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/arnoldm./nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:000:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [30]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# Load comments CSV
comment_df = pd.read_csv("../data/youtube_comment_data_cleaned.csv")

# Initialize Sentiment Analyzer
sid_obj = SentimentIntensityAnalyzer()

# List to store sentiment results
sentiment_results = []

def sentiment_scores():
    """Processes sentiment for each comment and stores results."""
    for _, row in comment_df.iterrows():
        comment_text = str(row["Comment"])  # Ensure text format
        comment_id = row["Index"]
        video_id = row["Id"]

        # Skip empty or NaN comments
        if comment_text.strip().lower() in ["nan", "none", ""]:
            continue

        # Compute sentiment scores
        sentiment_dict = sid_obj.polarity_scores(comment_text)
        compound_score = sentiment_dict["compound"]

        # Determine sentiment label
        if compound_score >= 0.05:
            sentiment_label = "Positive"
        elif compound_score <= -0.05:
            sentiment_label = "Negative"
        else:
            sentiment_label = "Neutral"

        # Append result to list
        sentiment_results.append({
            "Comment_ID": comment_id,
            "Comment": comment_text,
            "Sentiment_Score": compound_score,
            "Sentiment_Label": sentiment_label,
            "Video_Id": video_id
        })


if __name__ == "__main__":
    sentiment_scores()

    sentiment_results_df = pd.DataFrame(sentiment_results)
    sentiment_results_df.to_csv("../data/youtube_comment_sentiment.csv", index=False)

    print("✅ Sentiment analysis complete. Results saved to youtube_comment_sentiment.csv")


✅ Sentiment analysis complete. Results saved to youtube_comment_sentiment.csv


Question 1: What's the average sentiment per video?

In [ ]:
import pandas as pd
from collections import Counter

df = pd.read_csv("../data/youtube_comment_sentiment.csv")

sentiment_counts = df.groupby("Video_Id")["Sentiment_Label"].apply(lambda x: Counter(x).most_common(1)[0][0])

print(sentiment_counts)


Video_Id
-gYpCIbZjUQ    Positive
4OmC6LyO5QI    Positive
FJV7HeHT4q4    Positive
IKot8dQc1Ps    Positive
WEP5ubPMGDU    Positive
dJyz6iK8VXE    Positive
laSK7Pxh0_8    Positive
rhjiANJVR6g    Positive
wqEsTPaUZF0    Positive
yaXm6UsSHBM    Positive
Name: Sentiment_Label, dtype: object


In [36]:
sentiment_average = df.groupby("Video_Id")["Sentiment_Score"].mean()
print(sentiment_average)

Video_Id
-gYpCIbZjUQ    0.075570
4OmC6LyO5QI    0.240436
FJV7HeHT4q4    0.187170
IKot8dQc1Ps    0.164949
WEP5ubPMGDU    0.264401
dJyz6iK8VXE    0.234352
laSK7Pxh0_8    0.245622
rhjiANJVR6g    0.229627
wqEsTPaUZF0    0.166475
yaXm6UsSHBM    0.194000
Name: Sentiment_Score, dtype: float64
